<a href="https://colab.research.google.com/github/EdoardoMorucci/Plant-Leaves-Search-Engine---MIRCV/blob/main/model_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook describes the fine-tuning process of Convolutional Neural Network using as Base Network DenseNet

# Connection to Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Import

In [8]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt

# Data Preparation


The dataset is on Google Drive and the dataset directory has the structure:

```
dataset/
  class_1/
    image_1.jpg
    image_2.jpg
    ...
  class_2/
    image_3.jpg
    image_4.jpg
    ...
  ...
  ...
  class_n/
    ...
```

To train and test the model, we need three subsets: train, test and validation. To split the dataset, we use the [split-folder](https://pypi.org/project/split-folders/) package.

In [ ]:
!pip install split-folders tqdm

We need to check if the hardware accelaration is enabled, since training a CNN on a CPU could be infeasible.

In [ ]:
#check hardware acceleration
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


We define the costants with the directory of the dataset and the directory where the datasplits are created. In addition we define the image size and the batch size.

In [6]:
BASE_DIR = "gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine/"
DATA_DIR = '/content/Healthy-and-Unhealthy-Plants-Dataset-Segmented'
SETS_DIR = '/content/healthy-unhealthy-plants-sets'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 256


We need to create data splits. The dataset will be divided 80% in training set, 10% in validation set and 10% in test set.

In [ ]:
import splitfolders
# split data
splitfolders.ratio(DATA_DIR, output=SETS_DIR, seed=123, ratio=(0.8, 0.1, 0.1), group_prefix=None)

Copying files: 72034 files [00:18, 3871.66 files/s]


Now we need to create the Dataset objects from the sets directory. We use the [image_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory) function provided by Keras. An example of use of this library can be found on the official documentation provided by Keras ([here](https://keras.io/examples/vision/image_classification_from_scratch/)).

In [9]:

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    SETS_DIR + '/train',
    labels='inferred', #the label of the dataset is obtained by the name of the directory
    seed=123,
    shuffle=True,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    SETS_DIR + '/val',
    labels='inferred', #the label of the dataset is obtained by the name of the directory
    seed=123,
    shuffle=True,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    SETS_DIR + '/test',
    labels='inferred', #the label of the dataset is obtained by the name of the directory
    seed=123,
    shuffle=True,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)

Found 57623 files belonging to 14 classes.
Found 7195 files belonging to 14 classes.
Found 7216 files belonging to 14 classes.
